In [17]:
import torch 

In [18]:
import torch.nn as nn
import pandas as pd 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [19]:
torch.manual_seed(42)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [21]:
df = pd.read_csv('fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [24]:
# transformations 
from torchvision.transforms import transforms

custom_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]) ## normalized for VGG16 compatible data
])

In [25]:
from PIL import Image
import numpy as np 

class CustomDataset(Dataset):
    def __init__(self,features,labels,transforms):
        self.features = features
        self.labels = labels
        self.transforms = transforms
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        # reshape to size (28,28)
        image = self.features[index].reshape(28,28)
        # change datatype to np.uint8
        image = image.astype(np.uint8)
        # convert black&white(2D) to colored (3D)
        image = np.stack([image]*3,axis=-1)
        # convert array to PIL image 
        image = Image.fromarray(image)
        # apply transformations 
        image = self.transforms(image)

        return image , torch.tensor(self.labels[index],dtype=torch.long)
    

In [26]:
train_dataset = CustomDataset(X_train,y_train,transforms=custom_transforms)

In [27]:
test_dataset = CustomDataset(X_test,y_test,transforms = custom_transforms)

In [28]:
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [31]:
from torchvision.models import vgg16, VGG16_Weights

weights = VGG16_Weights.DEFAULT
model = vgg16(weights=weights)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\joshi/.cache\torch\hub\checkpoints\vgg16-397923af.pth


100%|██████████| 528M/528M [13:42<00:00, 673kB/s]  


In [32]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [33]:
for param in model.features.parameters():
    param.requires_grad = False

In [34]:
model.classifier = nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,10)
)

In [35]:
model = model.to(device)

In [36]:
learning_rate = 0.0001
epochs = 10

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr=learning_rate)

In [38]:
for epoch in range(epochs):
    total_epoch_loss = 0

    for batch_features , batch_labels in train_dataloader:
        batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
        outputs = model(batch_features)
        loss = criterion(outputs,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss = total_epoch_loss + loss.item()
        break
    avg_loss = total_epoch_loss/len(train_dataloader)
    print(f'Epoch : {epoch+1},Loss : {avg_loss}')


Epoch : 1,Loss : 0.001550256888071696
Epoch : 2,Loss : 0.0015169941584269205
Epoch : 3,Loss : 0.0014891366958618164
Epoch : 4,Loss : 0.0014275093078613282
Epoch : 5,Loss : 0.00138090451558431
Epoch : 6,Loss : 0.0014104183514912923
Epoch : 7,Loss : 0.0013069117069244385
Epoch : 8,Loss : 0.0013015758196512858
Epoch : 9,Loss : 0.001196886936823527
Epoch : 10,Loss : 0.0011283917427062989


In [39]:
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [40]:
total = 0
correct = 0
with torch.no_grad():
    for batch_features,batch_labels in test_dataloader:
        batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
    print(correct/total)

0.5375


In [41]:
total = 0
correct = 0
with torch.no_grad():
    for batch_features,batch_labels in train_dataloader:
        batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
    print(correct/total)

0.5366458333333334
